In [1]:
from torchvision import utils
from basic_fcn import *
from dataloader import *
from utils import *
import torchvision
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import time

In [2]:
train_dataset = CityScapesDataset(csv_file='./Data/train.csv')
val_dataset = CityScapesDataset(csv_file='./Data/val.csv')
test_dataset = CityScapesDataset(csv_file='./Data/test.csv')
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=1,
                          num_workers=4,
                          shuffle=True)
val_loader = DataLoader(dataset=val_dataset,
                          batch_size=1,
                          num_workers=4,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=3,
                          num_workers=4,
                          shuffle=True)

In [3]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform_(m.weight.data)
        #torch.nn.init.xavier_uniform(m.bias.data)
        m.bias.data.zero_()
        
epochs     = 100
criterion = nn.CrossEntropyLoss()# Choose an appropriate loss function from https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html
fcn_model = FCN(n_class=n_class)
# for name, params in fcn_model.named_parameters():
#     print(name, params.shape, params.requires_grad)
    
fcn_model.apply(init_weights)
#fcn_model = torch.load('best_model')


optimizer = optim.Adam(fcn_model.parameters(), lr=5e-3)

In [4]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    fcn_model = fcn_model.cuda()
    
def train():
    for epoch in range(epochs):
        ts = time.time()
        for iter, (X, tar, Y) in enumerate(train_loader):
            optimizer.zero_grad()

            if use_gpu:
                inputs = X.cuda()   # Move your inputs onto the gpu
                labels = Y.cuda() # Move your labels onto the gpu
            else:
                inputs, labels = X, Y# Unpack variables into inputs and labels
            outputs = fcn_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if iter % 10 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item(), time.time() - ts))
        
        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        torch.save(fcn_model, 'best_model')

        val(epoch)
        fcn_model.train()


def val(epoch):
    fcn_model.eval()
    #Complete this function - Calculate loss, accuracy and IoU for every epoch
    # Make sure to include a softmax after the output from your model
    for iter, (X_val, tar_val, Y_val) in enumerate(val_loader):
        if use_gpu:
            inputs_val = X_val.cuda()   # Move your inputs onto the gpu
            labels_val = Y_val.cuda() # Move your labels onto the gpu
        else:
            inputs_val, labels_val = X_val, Y_val# Unpack variables into inputs and labels

        outputs_val = fcn_model(inputs_val)
        
        _, preds = torch.max(outputs_val, 1)
        preds = preds.cuda()
        print('preds', preds)
        print(preds.size())
        loss_val = criterion(outputs_val, labels_val)
        print("Validation loss after epoch {}, , loss: {}".format(epoch, loss_val.item()))
        print("Accuracy on val after epoch {}, accuracy: {}".format(epoch, pixel_acc(preds, Y_val)))
        print("IOU on val after epoch{}".format(epoch, iou(outputs_val, Y_val)))
           
        
        #Add accuracy and IoU

def test():
    #Complete this function - Calculate accuracy and IoU 
    # Make sure to include a softmax after the output from your model
    for iter, (X_tst, tar_tst, Y_tst) in enumerate(test_loader):
        if use_gpu:
            inputs_tst = X_tst.cuda()   # Move your inputs onto the gpu
            labels_tst = Y_tst.cuda() # Move your labels onto the gpu
        else:
            inputs_tst, labels_tst = X_tst, Y_tst# Unpack variables into inputs and labels

        outputs_tst = fcn_model(inputs_tst)
        loss_tst = criterion(outputs_tst, labels_tst)
        print("Validation loss after epoch {}, , loss: {}".format(epoch, loss_tst.item()))
        #print("Accuracy on test after epoch {}, accuracy: {}".format(epoch, pixel_acc(outputs_tst, Y_tst)))
        #print("IOU on test after epoch{}".format(epoch, iou(outputs_tst, Y_tst)))
            
        
        #Add accuracy and IoU
              
    
if __name__ == "__main__":
    val(0)  # show the accuracy before training
    train()

preds tensor([[[ 7, 24,  7,  ..., 24,  7,  7],
         [ 2, 24, 24,  ..., 31,  0, 24],
         [ 7, 24,  7,  ..., 24, 27,  8],
         ...,
         [20, 26,  0,  ..., 24,  0, 24],
         [ 7, 17,  7,  ...,  7, 24, 24],
         [ 3, 24, 24,  ..., 24,  2, 20]]], device='cuda:0')
torch.Size([1, 1024, 2048])
Validation loss after epoch 0, , loss: 3.526348829269409
Accuracy on val after epoch 0, accuracy: 0.09196710586547852


NameError: name 'iou' is not defined